# Networking (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/Runtime.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> printException}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 60
|> _equal (Some false)

16:03:51 #1 [Verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 60 65535
|> Async.RunSynchronously
|> _equal false

16:03:51 #2 [Debug] runWithTimeoutAsync / timeout: 60
False


## waitForPortAccess

In [ ]:
let waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Warn (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 2 else 2)
            (if Runtime.isWindows () then 5 else 1500)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 80 else 80)
            (if Runtime.isWindows () then 150 else 600)

        true
    | _ -> false
|> _equal true

16:03:52 #3 [Debug] 1
16:03:52 #4 [Debug] _1
16:03:52 #5 [Debug] 2
16:03:52 #6 [Debug] 3
16:03:54 #7 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
16:03:54 #8 [Warn] waitForPortAccess / port: 5000 / retry: 0
16:03:56 #9 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
16:03:57 #10 [Debug] _2
16:03:57 #11 [Debug] _3
16:03:57 #12 [Debug] 4
16:03:57 #13 [Warn] waitForPortAccess / port: 5000 / retry: 0
16:03:58 #14 [Warn] waitForPortAccess / port: 5000 / retry: 100
16:03:59 #15 [Debug] _4
16:03:59 #16 [Debug] _5
16:04:01 #17 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
16:04:01 #18 [Debug] 5
16:04:01 #19 [Debug] 6
2
2
2
123
123
123
True


In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 4 else 2)
            (if Runtime.isWindows () then 15 else 150)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 5 else 3)
            (if Runtime.isWindows () then 20 else 60)

        true
    | _ -> false
|> _equal true

16:04:01 #20 [Debug] 1
16:04:01 #21 [Debug] 2
16:04:01 #22 [Debug] _1
16:04:01 #23 [Debug] 3
16:04:01 #24 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #25 [Warn] waitForPortAccess / port: 5000 / retry: 0
16:04:01 #26 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #27 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #28 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #29 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #30 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #31 [Debug] _2
16:04:01 #32 [Debug] _3
16:04:01 #33 [Debug] runWithTimeoutAsync / timeout: 60
16:04:01 #34 [Debug] 4
16:04:01 #35 [Warn] waitForPortAccess / port: 5000 / retry: 0
16:04:01 #36 [Debug] _4
16:04:01 #37 [Debug] _5
16:04:02 #38 [Debug] runWithTimeoutAsync / timeout: 60
16:04:02 #39 [Debug] 5
16:04:02 #40 [Debug] 6
7
7
7
11
11
11
True


## getAvailablePort

In [ ]:
let getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 60
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 1200)

        true
    | _ -> false
|> _equal true

16:04:02 #41 [Debug] 1
16:04:02 #42 [Debug] 2
16:04:02 #43 [Debug] _1
16:04:02 #44 [Debug] _2
16:04:02 #45 [Debug] _3
16:04:02 #46 [Debug] 3
16:04:04 #47 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
16:04:04 #48 [Debug] 4
16:04:04 #49 [Warn] waitForPortAccess / port: 5000 / retry: 0
16:04:05 #50 [Warn] waitForPortAccess / port: 5000 / retry: 100
16:04:06 #51 [Debug] _4
16:04:06 #52 [Debug] _5
16:04:08 #53 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
16:04:08 #54 [Debug] 5
16:04:08 #55 [Debug] 6
5002
122
122
122
True


In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 20
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 10 else 10)
            (if Runtime.isWindows () then 30 else 120)

        true
    | _ -> false
|> _equal true

16:04:08 #56 [Debug] 1
16:04:08 #57 [Debug] 2
16:04:08 #58 [Debug] _1
16:04:08 #59 [Debug] _2
16:04:08 #60 [Debug] _3
16:04:08 #61 [Debug] 3
16:04:08 #62 [Debug] runWithTimeoutAsync / timeout: 60
16:04:08 #63 [Debug] 4
16:04:08 #64 [Warn] waitForPortAccess / port: 5000 / retry: 0
16:04:08 #65 [Debug] _4
16:04:08 #66 [Debug] _5
16:04:08 #67 [Debug] runWithTimeoutAsync / timeout: 60
16:04:08 #68 [Debug] 5
16:04:08 #69 [Debug] 6
5002
21
21
21
True
